In [11]:
import pandas as pd
import os
import numpy as np
import cv2
from tqdm import tqdm

## Load Data and Extract Color

In [5]:
df = pd.read_csv('resources/data.csv')
df.head()

,Label,Weight
0,0_5_1.png,158.9
1,1_5_1.png,154.5
2,2_5_1.png,149.7
3,3_5_1.png,148.4
4,4_5_1.png,147.5


In [12]:
# Folder containing images
FOLDER_PATH = "resources/images"  # Change this to your folder path
OUTPUT_FILE = "resources/color_stats.csv"

# Define color spaces and their channels
COLOR_SPACES = {
    "RGB": (None, ["R", "G", "B"]),
    "LAB": (cv2.COLOR_BGR2LAB, ["L", "A", "B"]),
    "HSV": (cv2.COLOR_BGR2HSV, ["H", "S", "V"]),
    "GRAY": (cv2.COLOR_BGR2GRAY, ["Gray"])
}

def extract_color(image, conversion_code):
    """ Convert image to specified color space and return mean & std per channel. """
    img = cv2.cvtColor(image, conversion_code) if conversion_code else image
    return np.concatenate([np.mean(img, axis=(0, 1)), np.std(img, axis=(0, 1))]).astype(float) if img.ndim == 3 else [np.mean(img), np.std(img)]

# Get list of image files
image_files = [f for f in os.listdir(FOLDER_PATH) if f.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff'))]

# Process images with a progress bar
data = []
for file_name in tqdm(image_files, desc="Processing Images", unit="image"):
    image = cv2.imread(os.path.join(FOLDER_PATH, file_name))
    if image is not None:
        row = [file_name] + [val for space, (conv, _) in COLOR_SPACES.items() for val in extract_color(image, conv)]
        data.append(row)

# Generate column names dynamically
columns = ["Filename"] + [f"{stat}_{space}_{ch}" for space, (_, chs) in COLOR_SPACES.items() for stat in ["Mean", "Std"] for ch in chs]

# Save results to CSV
pd.DataFrame(data, columns=columns).to_csv(OUTPUT_FILE, index=False)
print(f"\nColor statistics extraction complete! Data saved to {OUTPUT_FILE}")


Processing Images:   0%|          | 0/337 [00:00<?, ?image/s]

Processing Images:  44%|████▍     | 149/337 [08:54<11:14,  3.59s/image]


KeyboardInterrupt: 